                                                        EDA_user_reviews

In [2]:
### Importamos las librerias necesarias para la lectura 
import pandas as pd
import ast

file = './australian_user_reviews.json'

data = []
with open(file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            ### Usamos ast.literal_eval para interpretar la línea como un diccionario de Python
            data_dict = ast.literal_eval(line)
            data.append(data_dict)
        except ValueError as e:
            print(f"Error al interpretar la línea: {e}")
            break  

# Convierte la lista de diccionarios a un DataFrame
df = pd.DataFrame(data)

In [3]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
### Función para extraer información de un diccionario en la lista 'reviews'
def extract_review_data(review_list):
    
    if not review_list:
        return pd.Series([None] * 7)
    
    # Extraer datos del primer diccionario de la lista
    review_dict = review_list[0]
    funny = review_dict.get('funny', '')
    posted = review_dict.get('posted', '').replace('Posted ', '')  # Eliminar "Posted "
    last_edited = review_dict.get('last_edited', '')
    item_id = review_dict.get('item_id', '')
    helpful = review_dict.get('helpful', '')
    recommend = review_dict.get('recommend', False)
    review = review_dict.get('review', '')

    return pd.Series([funny, posted, last_edited, item_id, helpful, recommend, review])

# Aplicar la función y crear nuevas columnas
columns = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review_content']
df[columns] = df['reviews'].apply(extract_review_data)

In [5]:
df.info()
df.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         25799 non-null  object
 1   user_url        25799 non-null  object
 2   reviews         25799 non-null  object
 3   funny           25771 non-null  object
 4   posted          25771 non-null  object
 5   last_edited     25771 non-null  object
 6   item_id         25771 non-null  object
 7   helpful         25771 non-null  object
 8   recommend       25771 non-null  object
 9   review_content  25771 non-null  object
dtypes: object(10)
memory usage: 2.0+ MB


,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review_content
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",,February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",,"October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...


                                 ETL:    CREAMOS LA COLUMNA  QUE SE NOS PIDE: SENTIMENT ANALISIS

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
import pandas as pd
from textblob import TextBlob
import nltk

# Descargar recursos necesarios para nltk
nltk.download('vader_lexicon')

# (Cargar o crear DataFrame df según el código proporcionado)

# Función para análisis de sentimiento
def analyze_sentiment(text):
    if pd.isnull(text):
        return 1  # Valor 1 si la reseña está ausente
    else:
        # Utilizar TextBlob para análisis de sentimiento
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity

        # Escalar el sentimiento a la escala deseada
        if sentiment < 0:
            return 0  # Valor 0 para sentimiento negativo
        elif sentiment == 0:
            return 1  # Valor 1 para sentimiento neutral
        else:
            return 2  # Valor 2 para sentimiento positivo

# Crear la nueva columna 'sentiment_analysis'
df['sentiment_analysis'] = df['review_content'].apply(analyze_sentiment)

# Mostrar información del DataFrame con la nueva columna
print("Información del DataFrame con la nueva columna 'sentiment_analysis':")
print(df[['review_content', 'sentiment_analysis']].head())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Información del DataFrame con la nueva columna 'sentiment_analysis':
                                      review_content  sentiment_analysis
0  Simple yet with great replayability. In my opi...                   2
1  I know what you think when you see this title ...                   2
2  A suitably punishing roguelike platformer.  Wi...                   2
3  This game... is so fun. The fight sequences ha...                   2
4                                            Git gud                   1


In [8]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25799 non-null  object
 1   user_url            25799 non-null  object
 2   reviews             25799 non-null  object
 3   funny               25771 non-null  object
 4   posted              25771 non-null  object
 5   last_edited         25771 non-null  object
 6   item_id             25771 non-null  object
 7   helpful             25771 non-null  object
 8   recommend           25771 non-null  object
 9   review_content      25771 non-null  object
 10  sentiment_analysis  25799 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 2.2+ MB


,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review_content,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",,February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",,"October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",3 people found this review funny,"April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud,1


In [11]:
import pandas as pd
import ast
from textblob import TextBlob
from ydata_profiling import ProfileReport

# Descargar recursos necesarios para nltk
nltk.download('vader_lexicon')

# Cargar datos desde el archivo australian_user_reviews.json
file_reviews = './australian_user_reviews.json'
data_reviews = []

with open(file_reviews, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data_dict = ast.literal_eval(line)
            data_reviews.append(data_dict)
        except ValueError as e:
            print(f"Error al interpretar la línea: {e}")
            break

# Crear DataFrame
df_reviews = pd.DataFrame(data_reviews)

# Función para extraer información de los reviews
def extract_review_data(review_list):
    if not review_list:
        return pd.Series([None] * 7)

    review_dict = review_list[0]
    funny = review_dict.get('funny', '')
    posted = review_dict.get('posted', '').replace('Posted ', '')
    last_edited = review_dict.get('last_edited', '')
    item_id = review_dict.get('item_id', '')
    helpful = review_dict.get('helpful', '')
    recommend = review_dict.get('recommend', False)
    review_content = review_dict.get('review', '')

    return pd.Series([funny, posted, last_edited, item_id, helpful, recommend, review_content])

# Aplicar la función y crear nuevas columnas
columns_reviews = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review_content']
df_reviews[columns_reviews] = df_reviews['reviews'].apply(extract_review_data)

# Función para análisis de sentimiento
def analyze_sentiment(text):
    if pd.isnull(text):
        return 1  # Valor 1 si la reseña está ausente
    else:
        # Utilizar TextBlob para análisis de sentimiento
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity

        # Escalar el sentimiento a la escala deseada
        if sentiment < 0:
            return 0  # Valor 0 para sentimiento negativo
        elif sentiment == 0:
            return 1  # Valor 1 para sentimiento neutral
        else:
            return 2  # Valor 2 para sentimiento positivo

# Crear la nueva columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review_content'].apply(analyze_sentiment)

# Ahora, puedes continuar con el código para el informe con ydata_profiling
# Reemplazar '?' con NaN en todo el DataFrame (si es necesario)
df_reviews.replace('?', pd.NA, inplace=True)

# Generar el informe
profile_reviews = ProfileReport(df_reviews)

# Guardar el informe en un archivo HTML
ruta_informe_reviews = "C:\\Users\\ASUS\\PI_ML_OPS\\PI_ML_OPS-\\INFORMES GENERADOS PARA EL DATASET\\informe_australian_user_reviews.html"
profile_reviews.to_file(ruta_informe_reviews)

print(f"Informe guardado exitosamente en: {ruta_informe_reviews}")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Informe guardado exitosamente en: C:\Users\ASUS\PI_ML_OPS\PI_ML_OPS-\INFORMES GENERADOS PARA EL DATASET\informe_australian_user_reviews.html


                                                ALERTA DEL INFORME ANTERIOR:
El conjunto de datos tiene 308 (1,2%) filas duplicadas:                     Duplicados
reviews es un tipo no compatible, verificar si necesita limpieza o análisis adicional: sin soporte.

                             ETL  COFORME EL DIAGNOSTICO DEL INFORME GENERADO 

In [25]:
import pandas as pd
import ast

# ... (código para cargar y procesar los datos)

# Eliminar filas duplicadas basándote en una combinación de columnas
subset_columns = ['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review_content']
df_reviews_cleaned = df_reviews.drop_duplicates(subset=subset_columns)

# Eliminar filas con todos los valores nulos en todas las columnas
df_reviews_cleaned = df_reviews_cleaned.dropna(how='all')

# Comprobación y conversión de tipos de datos
if df_reviews_cleaned['reviews'].dtype == 'object':
    df_reviews_cleaned['reviews'] = df_reviews_cleaned['reviews'].astype(str)

# Mostrar información del nuevo DataFrame limpio
print("Información del nuevo DataFrame después de la limpieza:")
print(df_reviews_cleaned.info())


Información del nuevo DataFrame después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25458 entries, 0 to 25798
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             25458 non-null  object
 1   user_url            25458 non-null  object
 2   reviews             25458 non-null  object
 3   funny               25457 non-null  object
 4   posted              25457 non-null  object
 5   last_edited         25457 non-null  object
 6   item_id             25457 non-null  object
 7   helpful             25457 non-null  object
 8   recommend           25457 non-null  object
 9   review_content      25457 non-null  object
 10  sentiment_analysis  25458 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 2.3+ MB
None


In [28]:
df.head(5)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review_content,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...",,February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...",,"October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...",3 people found this review funny,"April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud,1


In [31]:
import pandas as pd
import ast
import webbrowser
import json

# ... (código para cargar y procesar los datos)

# Crear un nuevo DataFrame con las columnas seleccionadas
columnas_seleccionadas = ['user_id', 'reviews', 'recommend', 'review_content', 'sentiment_analysis']
df_combinado = df_reviews_cleaned[columnas_seleccionadas]

# Guardar el nuevo DataFrame en formato JSON en la ruta especificada
ruta_guardado = "C:\\Users\\ASUS\\PI_ML_OPS\\PI_ML_OPS-\\dataAPI\\dataframe_user_reviews_combinado.json"
df_combinado.to_json(ruta_guardado, orient='records', lines=True)

# Utilizar webbrowser para abrir el archivo JSON en el navegador
webbrowser.open(ruta_guardado)


True